In [1]:
# %pip install sqlalchemy
# %pip install requests
# %pip install bs4

In [1]:
from DataObjects.article import Article
from DataObjects.legislation import Legislation
from DataObjects.public_consultation import PublicConsultation
from DataObjects.base import Base

import requests

from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from bs4 import BeautifulSoup
import time

Utility Functions

In [2]:
import re
from datetime import datetime
def ends_with_digit_of_variable_length(s:str):
    return bool(re.search(r'\?p=\d{1,4}$', s))

def convert_greek_date(date_str):
    months = {
        'Ιανουαρίου': 'January',
        'Φεβρουαρίου': 'February',
        'Μαρτίου': 'March',
        'Απριλίου': 'April',
        'Μαΐου': 'May',
        'Ιουνίου': 'June',
        'Ιουλίου': 'July',
        'Αυγούστου': 'August',
        'Σεπτεμβρίου': 'September',
        'Οκτωβρίου': 'October',
        'Νοεμβρίου': 'November',
        'Δεκεμβρίου': 'December'
    }

    for greek_month, english_month in months.items():
        date_str = date_str.replace(greek_month, english_month)

    return datetime.strptime(date_str, '%d %B %Y, %H:%M')

Regex Functions

In [3]:
def extract_article_number(text:str)->int:
    article_number = re.search(r"ρθρο (\d+)", text)
    if article_number is None:
        return 999
    return article_number.group(1)

Import Configuration

In [4]:
from configparser import ConfigParser

config = ConfigParser()
config.read("config.ini")


['config.ini']

In [5]:
import logging
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)

# Create a FileHandler and set its properties
file_handler = logging.FileHandler('example.log')
file_handler.setLevel(logging.DEBUG)

# Create a formatter to specify the log message format
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Add the FileHandler to the logger
logger.addHandler(file_handler)

In [6]:
URL = "http://www.opengov.gr/home/category/consultations/page/"

Scrap Legislation, Articles and public Consulations

In [7]:
engine = create_engine(config.get('DEFAULT', 'db_file'))
Base.metadata.create_all(engine)

log:
17-8-2023
scrapped til:

--------------------------------------------------------------------------------

http://www.opengov.gr/home/category/consultations/page/42

--------------------------------------------------------------------------------
http://www.opengov.gr/minenv/?p=4391

In [8]:
i=0
for _page in range(43,55):
    time.sleep(1)
    
    print(f"{URL}{_page}")
    request_url = f"{URL}{_page}"
    ConsultationPageResponse:requests.Response
    ConsultationPageResponse = requests.get(request_url)
    
    ConsultationResponseSoup = BeautifulSoup(ConsultationPageResponse.content, features="html.parser")

    legislationLinkTags = ConsultationResponseSoup.select("div.archive_list > ul > li a")

    print("--"*40)
    for tag in legislationLinkTags:
        consultationUrl = tag.get('href')
        if not ends_with_digit_of_variable_length(consultationUrl):  # Not a valid Link
            continue
        if "consultation" in consultationUrl:  # Not a Legislation
            continue

        print(tag.get('href'))
        inLegislationResponse = requests.get(tag.get('href'))
        inLegislationSoup = BeautifulSoup(inLegislationResponse.content, features="html.parser")
        # Collect Data
        data = {}
        try:
            txt_to_avoid = "Ολοκλήρωση της Δημόσιας Διαβούλευσης"
            titles = inLegislationSoup.select('h3')
            title = filter(lambda x: len(x.text)>len(txt_to_avoid), titles)
            title = list(title)[0].text

        except AttributeError as e:
            logger.debug(f"Cannot fetch Legislation ttile from {tag.get('href')}")
            continue
            
        data["title"] = title
        date_posted = inLegislationSoup.select_one('h4 span').text if inLegislationSoup.select_one('h4 span') is not None else None
        date_posted = convert_greek_date(date_posted) if date_posted is not None else None
        data["date_posted"] = date_posted
        ministry = inLegislationSoup.select_one('h1').text
        data["ministry"] = ministry
        legislation_pdf_url = inLegislationSoup.select_one("span.file a").get('href') if inLegislationSoup.select_one("span.file") is not None else None
        data["legislation_pdf_url"] = legislation_pdf_url
        data["scrap_url"] = tag.get('href')
        legislationObj = Legislation(**data)


        article_urls = inLegislationSoup.select("a.list_comments_link")
        articles_arr = []
        for a in article_urls:
            data = {}

            articleRequest = requests.get(a.get('href'))
            articleSoup = BeautifulSoup(articleRequest.content, features="html.parser")
            
            # Collect Data
            number = articleSoup.select_one('h3').text if articleSoup.select_one('h3') is not None else None
            number = extract_article_number(number)
            data["number"] = number
            
            title = articleSoup.select_one('h3').text if articleSoup.select_one('h3') is not None else None
            data["title"] = title
            
            text = articleSoup.select_one('div.post_content').text if articleSoup.select_one('div.post_content') is not None else None
            data["text"] = text

            comments_text = a.parent.select_one("span.list_comments").text if a.parent.select_one("span.list_comments") is not None else ""
            comments_not_allowed = True if "δεν επιτρ" in comments_text.lower() else False
            data['comments_allowed'] = not comments_not_allowed

            data["article_url"] = a.get("href")

            articleObj = Article(**data)
            articleObj.legislation = legislationObj
            articles_arr.append(articleObj)
            
           

            public_consulation_arr = []

            comments = articleSoup.select("li.comment")
            # Scap Public Consulations for Article

            for i in range(len(comments)) :
                
                comment = comments[i]
                public_consulation_data = {}
                
                reporter = comment.select_one("div.author strong").text if comment.select_one("div.author strong") is not None else "None"
                
                
                public_consulation_data["reporter"] = reporter
                
                
                date_reported = comment.select_one("div.author").text
                pipe_idx = date_reported.index("|")
                date_reported = date_reported[:pipe_idx].strip()
                date_reported = convert_greek_date(date_reported)
                public_consulation_data["date_reported"] = date_reported
                

                url = comment.select_one("div.meta-comment a").get('href') if comment.select_one("div.meta-comment a") is not None else None
                public_consulation_data["url"] = url
                
                text = comment.select("p")
                text = " ".join(p_tag.text for p_tag in text)
                public_consulation_data["text"] = text

                publicConsultationObj = PublicConsultation(**public_consulation_data)
                publicConsultationObj.articles = articleObj
                public_consulation_arr.append(publicConsultationObj)

            prev_tag = articleSoup.select_one("a.prev")
            # go to previous page and scrap comments
            while prev_tag:
                comments_response = requests.get(f"http://www.opengov.gr{prev_tag.get('href')}")
                articleSoup2 = BeautifulSoup(comments_response.content, features="html.parser")

                comments2 = articleSoup2.select("li.comment")
                for comment in comments2:

                    public_consulation_data = {}
            
                    reporter = comment.select_one("div.author strong").text if comment.select_one("div.author strong") is not None else "None"
                    public_consulation_data["reporter"] = reporter
                    
                    date_reported = comment.select_one("div.author").text
                    pipe_idx = date_reported.index("|")
                    date_reported = date_reported[:pipe_idx].strip()
                    date_reported = convert_greek_date(date_reported)
                    public_consulation_data["date_reported"] = date_reported
                    
                    url = comment.select_one("div.meta-comment a").get('href') if comment.select_one("div.meta-comment a") is not None else None
                    public_consulation_data["url"] = url
                    
                    text = comment.select("p")
                    text = " ".join(p_tag.text for p_tag in text)
                    public_consulation_data["text"] = text

                    publicConsultationObj = PublicConsultation(**public_consulation_data)
                    publicConsultationObj.articles = articleObj
                    public_consulation_arr.append(publicConsultationObj)
                
                prev_tag = articleSoup2.select_one("a.prev")



        with Session(engine) as sess:
            sess.add(legislationObj)
            sess.add_all(articles_arr)
            sess.add_all(public_consulation_arr)
            sess.commit()
           


        
            

    print("--"*40)
    


print("Done!")


http://www.opengov.gr/home/category/consultations/page/43
--------------------------------------------------------------------------------
http://www.opengov.gr/ministryofjustice/?p=1830
http://www.opengov.gr/minenv/?p=3731
http://www.opengov.gr/yme/?p=2303
http://www.opengov.gr/ypepth/?p=1436
http://www.opengov.gr/minenv/?p=3531
http://www.opengov.gr/minenv/?p=3494
http://www.opengov.gr/minreform/?p=178
http://www.opengov.gr/ypepth/?p=1128
http://www.opengov.gr/minenv/?p=3423
http://www.opengov.gr/ypepth/?p=1345
http://www.opengov.gr/minenv/?p=3354
http://www.opengov.gr/minenv/?p=3277
http://www.opengov.gr/minreform/?p=129
http://www.opengov.gr/tourism/?p=503
http://www.opengov.gr/minenv/?p=3203
http://www.opengov.gr/minenv/?p=3174
http://www.opengov.gr/yptp/?p=652
http://www.opengov.gr/tourism/?p=421
http://www.opengov.gr/minfin/?p=973
--------------------------------------------------------------------------------
http://www.opengov.gr/home/category/consultations/page/44
-----------

should check http://www.opengov.gr/mindefence/?p=7002 / not Legislation

In [ ]:
"consultation" in "http://www.opengov.gr/home/category/consultations/page/52"

True

In [30]:
comment.select_one('div.author').text

'\r\t\t 17 Ιουλίου 2023, 05:29 |  \r\t\t \t\t\t\t Αναστασία '

In [ ]:
r = requests.get("http://www.opengov.gr/ypes/?p=8812")
s = BeautifulSoup(r.content, features="html.parser")


In [ ]:
for comment in s.select("li.comment"):
    comments = comment.select("p")
    txt = " ".join(p_tag.text for p_tag in comments)
    # print(len(comment.select("p")))
    print(txt)
    print("**"*40)

Πολύ σημαντική πρωτοβουλία της κυβέρνησης, όλοι οι Έλληνες εγγεγραμμένοι στους εκλογικούς καταλόγους έχουν το δικαίωμα να ψηφίσουν.
Ακόμη σημαντικότερο όμως είναι να προβλεφθούν ενναλακτικοί τρόποι άσκησης του δικαιώματος αυτού μια και η διασπορά είναι μεγάλη και όπου δεν προβλέπονται εκλογικά τμήματα δε μπορούμε να μετακινούμαστε σε άλλη πόλη ή χώρα.
********************************************************************************
Η Ομοσπονδία Ελληνικών Κοινοτήτων στην Γερμανία χαιρετίζει την πρωτοβουλία αλλαγής του ν. 4648 του 2019, συμμετέχει στην διαβούλευση, στην συζήτηση και στον δημόσιο διάλογο και καταθέτει εδώ τις θέσεις της, όπως έκανε και στο παρελθόν. Θέσεις, οι οποίες έχουν διατυπωθεί εδώ και μερικές δεκαετίες.  Η ΟΕΚ θεωρεί κατ’ αρχήν ιδιαίτερα θετικό το γεγονός, ότι έστω και με αυτούς τους άδικους και απαράδεκτους όρους και γραφειοκρατικούς περιορισμούς, που προβλέπονται στον ισχύοντα νόμο, στις εκλογές του Μαΐου και του Ιουνίου 2023, μπόρεσε και άσκησε το εκλογικό του δι

In [ ]:
s.select("div.author")[2].text

'17 Ιουλίου 2023, 02:26 |  \r\t\t \t\t\t\t Aθηνα Βασιλειάδου'

In [ ]:
for c in articleSoup.select("li.comment"):
    # print(c.select_one("div.author").text if c.select_one("div.author") is not None else "")
    # print(c.select_one("div.user"))
    articleSoup.select("")

		 				 <strong>Αναστασία</strong> </div> <div class="meta-comment"> <a class="permalink" href="http://www.opengov.gr/ypes/?c=72367" title="">Μόνιμος Σύνδεσμος</a> <div class="rate"> </div> </div> </div> <p>Πολύ σημαντική πρωτοβουλία της κυβέρνησης, όλοι οι Έλληνες εγγεγραμμένοι στους εκλογικούς καταλόγους έχουν το δικαίωμα να ψηφίσουν.<br/>
Ακόμη σημαντικότερο όμως είναι να προβλεφθούν ενναλακτικοί τρόποι άσκησης του δικαιώματος αυτού μια και η διασπορά είναι μεγάλη και όπου δεν προβλέπονται εκλογικά τμήματα δε μπορούμε να μετακινούμαστε σε άλλη πόλη ή χώρα.</p>
</li>
		 				 <strong><a href="http://www.oekg.de" rel="nofollow" target="_blank">Πάνος Δροσινάκης</a></strong> </div> <div class="meta-comment"> <a class="permalink" href="http://www.opengov.gr/ypes/?c=72366" title="">Μόνιμος Σύνδεσμος</a> <div class="rate"> </div> </div> </div> <p>Η Ομοσπονδία Ελληνικών Κοινοτήτων στην Γερμανία χαιρετίζει την πρωτοβουλία αλλαγής του ν. 4648 του 2019, συμμετέχει στην διαβούλευση, στην συζήτησ

In [24]:
comments = articleSoup.select("li.comment")
# for i in range(len(comments)):
#     comment = comments[i]
#     print(comment.select_one("div.author").text)


[<li class="comment even thread-even depth-1" id="comment-72367"> <div class="user"> <div class="author">
 		 17 Ιουλίου 2023, 05:29 |  
 		 				 <strong>Αναστασία</strong> </div> <div class="meta-comment"> <a class="permalink" href="http://www.opengov.gr/ypes/?c=72367" title="">Μόνιμος Σύνδεσμος</a> <div class="rate"> </div> </div> </div> <p>Πολύ σημαντική πρωτοβουλία της κυβέρνησης, όλοι οι Έλληνες εγγεγραμμένοι στους εκλογικούς καταλόγους έχουν το δικαίωμα να ψηφίσουν.<br/>
 Ακόμη σημαντικότερο όμως είναι να προβλεφθούν ενναλακτικοί τρόποι άσκησης του δικαιώματος αυτού μια και η διασπορά είναι μεγάλη και όπου δεν προβλέπονται εκλογικά τμήματα δε μπορούμε να μετακινούμαστε σε άλλη πόλη ή χώρα.</p>
 </li>,
 <li class="comment odd alt thread-odd thread-alt depth-1" id="comment-72366"> <div class="user"> <div class="author">
 		 17 Ιουλίου 2023, 02:31 |  
 		 				 <strong><a href="http://www.oekg.de" rel="nofollow" target="_blank">Πάνος Δροσινάκης</a></strong> </div> <div class="meta-co

In [ ]:
for a in articleSoup.select("li.comment"):
    print(a.text)

		 				 Αναστασία   Μόνιμος Σύνδεσμος     Πολύ σημαντική πρωτοβουλία της κυβέρνησης, όλοι οι Έλληνες εγγεγραμμένοι στους εκλογικούς καταλόγους έχουν το δικαίωμα να ψηφίσουν.
Ακόμη σημαντικότερο όμως είναι να προβλεφθούν ενναλακτικοί τρόποι άσκησης του δικαιώματος αυτού μια και η διασπορά είναι μεγάλη και όπου δεν προβλέπονται εκλογικά τμήματα δε μπορούμε να μετακινούμαστε σε άλλη πόλη ή χώρα.

		 				 Πάνος Δροσινάκης   Μόνιμος Σύνδεσμος     Η Ομοσπονδία Ελληνικών Κοινοτήτων στην Γερμανία χαιρετίζει την πρωτοβουλία αλλαγής του ν. 4648 του 2019, συμμετέχει στην διαβούλευση, στην συζήτηση και στον δημόσιο διάλογο και καταθέτει εδώ τις θέσεις της, όπως έκανε και στο παρελθόν. Θέσεις, οι οποίες έχουν διατυπωθεί εδώ και μερικές δεκαετίες. 
Η ΟΕΚ θεωρεί κατ’ αρχήν ιδιαίτερα θετικό το γεγονός, ότι έστω και με αυτούς τους άδικους και απαράδεκτους όρους και γραφειοκρατικούς περιορισμούς, που προβλέπονται στον ισχύοντα νόμο, στις εκλογές του Μαΐου και του Ιουνίου 2023, μπόρεσε και άσκησε το εκλο

In [18]:
articleSoup.select("li.comment")[0].select_one("div.author").text

'\r\t\t 17 Ιουλίου 2023, 05:29 |  \r\t\t \t\t\t\t Αναστασία '